In [1]:
# !pip install mtcnn
# !pip install tensorflow
# !pip install facenet-pytorch

Processing OULU-NPU into image frames for training

In [2]:
import os
import cv2
import shutil
import random
import string
import numpy as np
from tqdm.notebook import tqdm
# from imutils import face_utils
import matplotlib.pyplot as plt
# from mtcnn.mtcnn import MTCNN

In [3]:
# face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')  

In [4]:
# def extract_face(img):
#     # Convert to grayscale 
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
  
#     # Detect the faces  
#     faces = face_cascade.detectMultiScale(gray, 1.3, 4)  
    
#     # Draw rectangle around the faces and crop the faces
#     for (x, y, w, h) in faces:
#         faces = img[y:y + h, x:x + w]
        
#     return faces

In [5]:
# def extract_face(img):
#     # Detect faces
#     results = detector.detect_faces(img)
    
#     faces = []
#     for result in results:
#         x, y, w, h = result['box']
#         face = img[y:y+h, x:x+w]
#         faces.append(face)
    
#     return faces

In [6]:
# def extract_face(img):
#     face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')  
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
#     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
#     face_images = []
#     for (x, y, w, h) in faces:
#         face = img[y:y+h, x:x+w]
#         face_images.append(face)
    
#     return face_images

In [7]:
# def extract_face_and_save(image_path, output_path):
#     # Load the image
#     img = cv2.imread(image_path)
    
#     # Convert to grayscale
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
#     # Detect the faces
#     faces = face_cascade.detectMultiScale(gray, 1.3, 4)
    
#     # Draw rectangle around the faces and save the image
#     for (x, y, w, h) in faces:
#         cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw a green rectangle around the face
        
#     # Save the image with rectangles
#     cv2.imwrite(output_path, img)
    
#     return img

In [8]:
# def extract_face_and_save(image_path, output_path):
#     # Load the image
#     img = cv2.imread(image_path)
    
#     # Initialize the MTCNN detector
#     detector = MTCNN()
    
#     # Detect faces
#     results = detector.detect_faces(img)
    
#     # Draw rectangle around the faces and save the image
#     for result in results:
#         x, y, w, h = result['box']
#         cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw a green rectangle around the face
    
#     # Save the image with rectangles
#     cv2.imwrite(output_path, img)
    
#     return img

In [9]:
# extract_face_and_save('./test_img.jpg', 'test_img_output_face_bound.jpg')

In [10]:
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

def extract_face(img, eye_coords):
    # Get eye coordinates
    x_eye_left, y_eye_left, x_eye_right, y_eye_right = map(int, eye_coords)
    # print(x_eye_left, y_eye_left, x_eye_right, y_eye_right)
    # Calculate the approximate size of the face using the distance between the eyes
    face_width = abs(x_eye_left - x_eye_right) * 3.5  # Adjust factor as needed
    face_height = face_width * 1.2  # Adjust factor as needed

    # Calculate the center point between the eyes
    center_x = (x_eye_left + x_eye_right) // 2
    center_y = (y_eye_left + y_eye_right) // 2
    # Define bounding box around the face
    # print(int(center_x - face_width // 2), int(center_x + face_width // 2))
    # print(int(center_y - face_height // 2), int(center_y + face_height // 2))
    
    x1 = max(0, int(center_x - face_width // 2))
    y1 = max(0, int(center_y - face_height // 2))
    x2 = min(img.shape[1], int(center_x + face_width // 2))
    y2 = min(img.shape[0], int(center_y + face_height // 2))
    if x1 < x2 and y1 < y2:
        # Get the region around the eyes
        # print(x1, y1, x2, y2)
        eyes_region = img[y1:y2, x1:x2]
        # Detect faces in the eyes region
        gray_eyes = cv2.cvtColor(eyes_region, cv2.COLOR_BGR2GRAY)
        def try_detect(scale_factor, min_neighbors):
            try:
                faces = face_cascade.detectMultiScale(gray_eyes, scale_factor, min_neighbors)
                if len(faces) > 0:
                    (fx, fy, fw, fh) = faces[0]

                    fx1 = x1 + fx
                    fy1 = y1 + fy
                    fx2 = fx1 + fw
                    fy2 = fy1 + fh

                    face = img[fy1:fy2, fx1:fx2]

                    return face

            except Exception as e:
                pass

        # Try detecting with different parameters
        face = try_detect(1.2, 3)
        if face is None or face.shape[0] * face.shape[1] < 1e5:
            face = try_detect(1.1, 5)
        if face is None or face.shape[0] * face.shape[1] < 1e5:
            face = try_detect(1.1, 3)
        if face is None or face.shape[0] * face.shape[1] < 1e5:
            face = try_detect(1.3, 4)
        if face is None or face.shape[0] * face.shape[1] < 1e5:
            face = try_detect(1.5, 4)
        if face is not None:
            return face

    return None



def read_eye_coordinates(txt_file):
    with open(txt_file, 'r') as file:
        eye_coords = [line.strip().split(',') for line in file]

    return eye_coords

In [11]:
def extract_multiple_videos(input_filenames, output_path, resize):
    i = 0  # Counter of first video
    abnormal_total_count=0
    no_detections_total_count=0
    for input_filename in tqdm(input_filenames, desc='Processing Videos'):
        # print(f"\rFile: {i}")
        img_name = 'client'
        pattern = input_filename.split('/')
        last_substring = pattern[-1]  # Get the last substring after splitting by '/'
        # print(last_substring)
        f_name = last_substring.split('_')  # Split the last substring by underscores
        if len(f_name) >= 4:  # Ensure there are at least 4 parts in the split
            img_name = '_'.join(f_name[:3])  # Join the first three parts with underscores

        # print(img_name)  # Print or use img_name as needed

        # Get the eye coordinates for this video
        eye_coordinates = read_eye_coordinates(input_filename.replace('.avi', '.txt'))

        abnormal_count=0
        no_detection_count=0
        # Open video capture
        cap = cv2.VideoCapture(input_filename)
        sec_count = 0

        while True:
            ret, frame = cap.read()

            if ret:
                try:
                    eye_coords = eye_coordinates[sec_count]
                    cropped_faces = extract_face(frame, eye_coords[1:])  # Exclude num_frame
                    if cropped_faces is None:
                        # print('No detection')
                        sec_count +=1
                        no_detection_count +=1
                        continue
                    if cropped_faces.shape[0]*cropped_faces.shape[1] < 1e5:
                        # print('abnormal detection')
                        sec_count +=1
                        abnormal_count +=1
                        continue
                    cropped_faces = cv2.resize(cropped_faces, (resize, resize))
                    write_path = output_path + f'{img_name}_{eye_coords[0]}_{i}.png'  # Use num_frame
                    cv2.imwrite(write_path, cropped_faces)
                    i += 1
                    sec_count += 1
                except Exception as e:
                    print(f"Error processing frame {sec_count}: {e}")
                    sec_count += 1
            else:
                break


        cap.release()
        if abnormal_count!=0 or no_detection_count!=0:
            print(last_substring)
        if abnormal_count!=0:
            print(f'abnormal detections: {abnormal_count}')
        if no_detection_count!=0:
            print(f'no detections: {no_detection_count}')
        abnormal_total_count +=abnormal_count
        no_detections_total_count +=no_detection_count
    print(f'Total abnormal detections count: {abnormal_total_count}')
    print(f'Total No detections count: {no_detections_total_count}')

    return i


In [12]:
# # START CAPTURING VIDEOS

# def extract_multiple_videos(input_filenames,output_path,img_class,frame_rate, resize):
#     """Extract video files into sequence of images.
#        input_filenames is a list for video file names"""

#     i = 0  # Counter of first video

#     # Iterate file names:
#     for input_filename in input_filenames:
        
#         img_name = 'client'
#         pattern = input_filename.split('/')
#         last_substring = pattern[-1]  # Get the last substring after splitting by '/'
#         print(last_substring)
#         f_name = last_substring.split('_')  # Split the last substring by underscores
#         if len(f_name) >= 4:  # Ensure there are at least 4 parts in the split
#             img_name = '_'.join(f_name[:3])  # Join the first three parts with underscores
    
#         print(img_name)  # Print or use img_name as needed
        
#         # print('the image will be named as:', img_name)
        
        
#         sec_count = 0
#         cap = cv2.VideoCapture(input_filename)
#         cap.set(cv2.CAP_PROP_POS_MSEC,(sec_count*frame_rate))

#         # Keep iterating break
#         while True:
#             ret, frame = cap.read()  # Read frame from first video

#             if ret:
#                 try:
#                     cropped_faces = extract_face(frame)
#                     cropped_faces = cv2.resize(cropped_faces,(resize,resize))
#                     write_path = output_path + f'{img_name}_{i}_{img_class}.png'
#                     cv2.imwrite(write_path, cropped_faces)  # Write frame to JPEG file (1.jpg, 2.jpg, ...)
#                     i += 1 # Advance file counter
#                     sec_count += 1
#                 except:
#                     pass
#             else:
#                 # Break the interal loop when res status is False.
#                 break

#             delay = int(1000 / frame_rate)  # Calculate the delay based on the actual frame rate
#             cv2.waitKey(delay)

#         cap.release() #Release must be inside the outer loop
#     return i

In [13]:
# from facenet_pytorch import MTCNN
# import torch

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# mtcnn = MTCNN(keep_all=True, device=device)  # Initialize the detector outside the function

# def extract_face_using_eyes(img, eye_coords):
#     # Use eye coordinates to define region of interest (ROI) for face detection
#     x_eye_left, y_eye_left, x_eye_right, y_eye_right = eye_coords
    
#     # Define the region around the eyes for face detection
#     roi_x = min(x_eye_left, x_eye_right)
#     roi_y = min(y_eye_left, y_eye_right)
#     roi_w = max(x_eye_left, x_eye_right) - roi_x
#     roi_h = max(y_eye_left, y_eye_right) - roi_y
    
#     # Extract the region for face detection
#     roi = img[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]
    
#     # Detect faces in the ROI
#     boxes, probs = mtcnn.detect(roi)
    
#     # Adjust the face coordinates to the original image space
#     if boxes is not None:
#         boxes[:, [0, 2]] += roi_x
#         boxes[:, [1, 3]] += roi_y
    
#     return boxes

# def extract_multiple_videos_with_eyes(input_filenames, output_path, img_class, frame_rate, resize):
#     i = 0  # Counter of first video

#     for input_filename in input_filenames:
#         img_name = 'client'
#         pattern = input_filename.split('/')
#         last_substring = pattern[-1]
#         f_name = last_substring.split('_')
#         if len(f_name) >= 4:
#             img_name = '_'.join(f_name[:3])
    
#         sec_count = 0
#         cap = cv2.VideoCapture(input_filename)
#         cap.set(cv2.CAP_PROP_POS_MSEC, (sec_count * frame_rate))

#         while True:
#             ret, frame = cap.read()

#             if ret:
#                 try:
#                     # Read eye coordinates from the corresponding txt file
#                     eye_coords_filename = f'{input_filename[:-4]}.txt'
#                     with open(eye_coords_filename, 'r') as coords_file:
#                         lines = coords_file.readlines()
#                         if sec_count >= len(lines):
#                             break  # Stop processing frames if no more coordinates are available
#                         eye_coords = list(map(int, lines[sec_count].split(',')[1:]))

#                     # Use eye coordinates for face detection around the eyes
#                     face_results = extract_face_using_eyes(frame, eye_coords)

#                     # Draw rectangles around detected faces
#                     if face_results is not None:
#                         for box in face_results:
#                             x, y, w, h = box
#                             cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)

#                     # Save the frame with rectangles
#                     write_path = output_path + f'{img_name}_{i}_{img_class}.png'
#                     cv2.imwrite(write_path, frame)

#                     i += 1
#                     sec_count += 1

#                 except Exception as e:
#                     print(f"Error processing frame: {e}")

#             else:
#                 break

#             delay = int(1000 / frame_rate)
#             cv2.waitKey(delay)

#         cap.release()

#     return i


In [14]:
def start(in_dir , out_dir, img_class, frame_rate=32, resize=256):
    filescounter = os.listdir(in_dir)
    # Count the number of files
    num_files = len(filescounter)
    print(num_files)
    
    # traverse whole directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    out_dir_real= os.path.join(out_dir, 'real/')
    out_dir_attack= os.path.join(out_dir, 'attack/')
    if not os.path.exists(out_dir_real):
        os.makedirs(out_dir_real)
    if not os.path.exists(out_dir_attack):
        os.makedirs(out_dir_attack)
    
    real_dir_list = []
    attack_dir_list = []
    for root, dirs, files in os.walk(f'{in_dir}'):
        # select file name
        for file in files:
            # check the extension of files
            if file.endswith('.avi'):
                # print whole path of files
                base_name = os.path.splitext(file)[0]
                if base_name[-1:] == '1':
                    path = os.path.join(root, file)
                    real_dir_list.append(path)
                else:
                    path = os.path.join(root, file)
                    attack_dir_list.append(path)
    count_real = extract_multiple_videos(real_dir_list,f'{out_dir_real}', resize=resize)
    count_attack = extract_multiple_videos(attack_dir_list,f'{out_dir_attack}', resize=resize)
    print(f'TOTAL FRAMES/IMAGES FORMED (REAL): {count_real}')
    print(f'TOTAL FRAMES/IMAGES FORMED (ATTACK): {count_attack}')

In [15]:
# # START CAPTURING VIDEOS

# def extract_attack_multiple_videos(input_filenames,output_path,img_class,frame_rate, resize):
#     """Extract video files into sequence of images.
#        input_filenames is a list for video file names"""

#     i = 0  # Counter of first video

#     # Iterate file names:
#     for input_filename in input_filenames:
#         print("Processing video:", input_filename)
#         img_name = 'client'
#         pattern = input_filename.split('/')
#         for p in pattern:
#             if 'client' in p:
#                 f_name = p.split('_')
#                 for name in f_name:
#                     if name.find('client') != -1:
#                         img_name = name
        
#         # print('the image will be named as:', img_name)
        
        
#         sec_count = 0
#         cap = cv2.VideoCapture(input_filename)
#         cap.set(cv2.CAP_PROP_POS_MSEC,(sec_count*frame_rate))

#         # Keep iterating break
#         while True:
#             ret, frame = cap.read()  # Read frame from first video
#             if ret:
#                 try:
#                     cropped_faces = extract_face(frame)
#                     cropped_faces = cv2.resize(cropped_faces,(resize,resize))
#                     write_path = output_path + f'{img_name}_{i}_{img_class}.png'
#                     cv2.imwrite(write_path, cropped_faces)  # Write frame to JPEG file (1.jpg, 2.jpg, ...)
#                     i += 1 # Advance file counter
#                     sec_count += 1
#                 except:
#                     pass
#             else:
#                 # Break the interal loop when res status is False.
#                 break

#             # cv2.waitKey(100) #Wait 100msec (for debugging)

#         cap.release() #Release must be inside the outer loop
#     return i

In [16]:
# def start_attack(in_dir , out_dir,img_class,frame_rate=32,resize=256):
#     # traverse whole directory
#     if not os.path.exists(out_dir):
#         os.makedirs(out_dir)
#     dir_list = []
#     for root, dirs, files in os.walk(f'{in_dir}'):
#         # select file name
#         for file in files:
#             # check the extension of files
#             if file.endswith('.mov'):
#                 # print whole path of files
#                 path = os.path.join(root, file)
#                 # print(path)
#                 dir_list.append(path)
#                 # extractImages(path,'./data/train/real',frame_rate=15000)
#     count = extract_attack_multiple_videos(dir_list,f'{out_dir}',img_class,frame_rate=frame_rate, resize=resize)
#     print(f'TOTAL FRAMES/IMAGES FORMED: {count}')

### TRANSFORMING TRAINING VIDEOS TO TRAINING IMAGES

In [17]:
train_vids_path = '/home/dinesh/Documents/Datasets/Face anti-spoofing datasets/OULU-NPU/Train_files'
dev_vids_path = '/home/dinesh/Documents/Datasets/Face anti-spoofing datasets/OULU-NPU/Dev_files'
test_vids_path = '/home/dinesh/Documents/Datasets/Face anti-spoofing datasets/OULU-NPU/Test_files'

In [18]:
train_imgs_path = '/home/dinesh/Documents/Datasets/Face anti-spoofing datasets/OULU-NPU-processed/train/'
dev_imgs_path = '/home/dinesh/Documents/Datasets/Face anti-spoofing datasets/OULU-NPU-processed/dev/'
test_imgs_path = '/home/dinesh/Documents/Datasets/Face anti-spoofing datasets/OULU-NPU-processed/test/'

In [19]:
train_dir = {"in_dir": train_vids_path,"out_dir":train_imgs_path}
dev_dir = {"in_dir":dev_vids_path, 'out_dir':dev_imgs_path}
test_dir = {"in_dir":test_vids_path, 'out_dir':test_imgs_path}

In [20]:
start(train_dir['in_dir'],train_dir['out_dir'],img_class="train",frame_rate=30,resize=256) 

3600


Processing Videos:   0%|          | 0/360 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
start(dev_dir['in_dir'],dev_dir['out_dir'],img_class="dev",frame_rate=32,resize=256) 

2700


Processing Videos:   0%|          | 0/270 [00:00<?, ?it/s]

3_3_22_1.avi
1_2_31_1.avi
3_1_31_1.avi
5_2_26_1.avi
3_3_21_1.avi
5_1_21_1.avi
4_1_22_1.avi
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detection
abnormal detecti

KeyboardInterrupt: 

In [22]:
start(test_dir['in_dir'],test_dir['out_dir'],img_class="test",frame_rate=32,resize=256)

3600


Processing Videos:   0%|          | 0/360 [00:00<?, ?it/s]

1_3_46_1.avi
4_3_43_1.avi
6_2_47_1.avi
3_2_45_1.avi
3_1_44_1.avi
5_2_53_1.avi
1_1_48_1.avi
3_2_51_1.avi
5_2_51_1.avi
3_3_39_1.avi
5_3_54_1.avi
4_3_46_1.avi
5_3_44_1.avi
No detection
1_1_55_1.avi
abnormal detection
abnormal detection
abnormal detection
1_2_55_1.avi
3_1_37_1.avi
6_1_53_1.avi
4_1_37_1.avi
1_2_38_1.avi
6_3_36_1.avi
2_3_40_1.avi
4_1_47_1.avi
1_3_49_1.avi
1_2_49_1.avi
5_1_42_1.avi
4_2_39_1.avi
6_3_51_1.avi
6_3_54_1.avi
5_1_37_1.avi
5_1_53_1.avi
3_2_37_1.avi
6_1_37_1.avi
6_3_39_1.avi
2_2_41_1.avi
4_1_54_1.avi
3_3_42_1.avi
2_3_37_1.avi
5_3_45_1.avi
6_2_46_1.avi
2_1_55_1.avi


KeyboardInterrupt: 

### TRANSFORMING TEST VIDEOS TO TESTING IMAGES 

In [20]:
test_real_vids_path = 'D:/BTP/dataset/Replay-Attack/test/real/'
test_spoof_fixed_vids_path = 'D:/BTP/dataset/Replay-Attack/test/attack/fixed/'
test_spoof_hand_vids_path = 'D:/BTP/dataset/Replay-Attack/test/attack/hand/'

In [21]:
test_real_imgs_path = './data/Replay-Attack-processed/test/real/'
test_spoof_fixed_imgs_path = './data/Replay-Attack-processed/test/spoof/fixed/'
test_spoof_hand_imgs_path = './data/Replay-Attack-processed/test/spoof/hand/'

In [22]:
test_dir_real = {"in_dir": test_real_vids_path,"out_dir":test_real_imgs_path}
test_dir_attack_fixed = {"in_dir":test_spoof_fixed_vids_path, 'out_dir':test_spoof_fixed_imgs_path}
test_dir_attack_hand = {"in_dir":test_spoof_hand_vids_path, 'out_dir':test_spoof_hand_imgs_path}

In [15]:
start(test_dir_real['in_dir'],test_dir_real['out_dir'],img_class="real",frame_rate=2000,resize=256) 

TOTAL FRAMES/IMAGES FORMED: 29229


In [15]:
start(test_dir_attack_fixed['in_dir'],test_dir_attack_fixed['out_dir'],img_class="attack_fixed",frame_rate=500,resize=256) 

TOTAL FRAMES/IMAGES FORMED: 46700


In [15]:
start(test_dir_attack_hand['in_dir'],test_dir_attack_hand['out_dir'],img_class="attack_hand",frame_rate=500,resize=256) 

TOTAL FRAMES/IMAGES FORMED: 44487


In [4]:
import os
import random
import csv

# Define the paths to your train and test folders
train_folder = './data/Replay-Attack-processed/train/'
test_folder = './data/Replay-Attack-processed/test/'

# Function to label and shuffle data in a folder
def process_folder(folder_path, label):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.png'):
                image_path = os.path.join(root, file)
                image_paths.append((image_path, label))
    
    # Shuffle the image paths
    random.shuffle(image_paths)
    
    return image_paths

# Process the train and test folders
train_real_paths = process_folder(os.path.join(train_folder, 'real'), 1)
test_real_paths = process_folder(os.path.join(test_folder, 'real'), 1)

# Include images from the "hand" and "fixed" subfolders within the "spoof" folder
train_fake_paths = process_folder(os.path.join(train_folder, 'spoof/hand'), 0)
train_fake_paths += process_folder(os.path.join(train_folder, 'spoof/fixed'), 0)

test_fake_paths = process_folder(os.path.join(test_folder, 'spoof/hand'), 0)
test_fake_paths += process_folder(os.path.join(test_folder, 'spoof/fixed'), 0)

# Combine real and fake paths
train_data = train_real_paths + train_fake_paths
test_data = test_real_paths + test_fake_paths

# Shuffle the combined data
random.shuffle(train_data)
random.shuffle(test_data)

# Define CSV filenames
train_csv_filename = 'train_data.csv'
test_csv_filename = 'test_data.csv'

# Write data to CSV files
def write_to_csv(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['name', 'label'])
        csv_writer.writerows(data)

write_to_csv(train_csv_filename, train_data)
write_to_csv(test_csv_filename, test_data)

print(f"Train data saved to {train_csv_filename}")
print(f"Test data saved to {test_csv_filename}")


Train data saved to train_data.csv
Test data saved to test_data.csv


### CHANGE DIRECTORY STRUCTURE BY PLACING IMAGES PERSON WISE

In [32]:
person_wise_dataset_train_path = './data/Reply-Attack-processed/train/images/' 

In [15]:
def get_dir_list(path):
    dir_list = []
    for root, dirs, files in os.walk(f'{path}'):
        # select file name
        for file in files:
            # check the extension of files
            if file.endswith('.png'):
                # print whole path of files
                path = os.path.join(root, file)
                # print(path)
                dir_list.append(path)
    return dir_list

In [23]:
img_path_list_train_real = get_dir_list(train_real_imgs_path)
img_path_list_train_spoof_fixed = get_dir_list(train_spoof_fixed_imgs_path)
img_path_list_train_spoof_hand = get_dir_list(train_spoof_hand_imgs_path)
print(len(img_path_list_train_real)) # ./replay-images/train/real/
print(len(img_path_list_train_spoof_fixed)) # ./replay-images/train/spoof/fixed
print(len(img_path_list_train_spoof_hand)) # ./replay-images/train/spoof/hand

22490
35438
33707


In [24]:
img_path_list_train_real[0]

'./data/Reply-Attack-processed/train/real/client001_0_real.png'

In [25]:
def get_client_objects(path_list):
    clients = {}
    for file in path_list:
        path_parts = file.split('/')
        for part in path_parts:
            if 'client' in part:
                file_name_split = part.split('_')
                client_key = file_name_split[0]
                if client_key in clients.keys():
                    clients[client_key].append(file)
                else:
                    clients[client_key] = [file]
    return clients

In [26]:
real_client_objects =  get_client_objects(img_path_list_train_real)
spoof_fixed_client_objects = get_client_objects(img_path_list_train_spoof_fixed)
spoof_hand_client_objects = get_client_objects(img_path_list_train_spoof_hand)

In [27]:
print(sorted(real_client_objects.keys()))
print(sorted(spoof_fixed_client_objects.keys()))
print(sorted(spoof_hand_client_objects.keys()))

['client001', 'client002', 'client004', 'client006', 'client007', 'client008', 'client012', 'client016', 'client018', 'client025', 'client027', 'client103', 'client105', 'client108', 'client110']
['client001', 'client002', 'client004', 'client006', 'client007', 'client008', 'client012', 'client016', 'client018', 'client025', 'client027', 'client103', 'client105', 'client108', 'client110']
['client001', 'client002', 'client004', 'client006', 'client007', 'client008', 'client012', 'client016', 'client018', 'client025', 'client027', 'client103', 'client105', 'client108', 'client110']


In [33]:
person_wise_dir_list = get_dir_list(person_wise_dataset_train_path)

In [34]:
person_wise_dir_list

[]

In [ ]:
ls

 DataGeneration.ipynb                              replay-dummy-person-wise/
 dataset/                                          replay-images/
 haarcascade_frontalface_default.xml               test_img.png
'PIxel Wise Supervision - Siamese-Network.ipynb'


### CREATE list.txt AND APPEND FORMATED PATHS OF IMAGES TO IT

In [29]:
with open('./data/Reply-Attack-processed/train/list.txt', 'w') as f:
    for path in person_wise_dir_list:
        if '/data/Reply-Attack-processed/train/images/' in path:
            s = path.split('/data/Reply-Attack-processed/train/images/')
            p = s[1]
            f.write("%s\n" % p)

In [30]:
for client_key in sorted(real_client_objects.keys()):
    if os.path.isdir(person_wise_dataset_train_path+client_key):
        shutil.rmtree(person_wise_dataset_train_path+client_key)
    single_client_path = person_wise_dataset_train_path+client_key
    os.mkdir(single_client_path)
    
    single_client_path_real = single_client_path + '/real'
    single_client_path_spoof = single_client_path + '/spoof'
    os.mkdir(single_client_path_real)
    os.mkdir(single_client_path_spoof)
    for raw_client_image_path in real_client_objects[client_key]:
        shutil.copy2(raw_client_image_path,single_client_path_real)
    for raw_client_image_path in spoof_fixed_client_objects[client_key]:
        shutil.copy2(raw_client_image_path,single_client_path_spoof)
    for raw_client_image_path in spoof_hand_client_objects[client_key]:
        shutil.copy2(raw_client_image_path,single_client_path_spoof)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './data/Reply-Attack-processed/train/images/client001'